# Self-Contained Financial Analysis and Insights Notebook

This notebook contains all the necessary code to analyze financial data, categorize transactions, generate insights using AI, and create PDF reports. No external .py files are required.

## License

This notebook is licensed under the GPLv3 License. See the LICENSE file for more details.
- Copyright (C) 2024 [Vincent Koc](https://github.com/vincentkoc)
- Copyright (C) 2024 [Hung Nguyen](https://github.com/hung-ngm)
## Setup

First, let's import the necessary libraries and set up our environment.

In [1]:
%pip install python-dotenv pandas gspread oauth2client openai==0.28.0 pdfkit gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.7/322.7 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 101.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.8 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.76.0
    Uninstalling openai-1.76.0:
      Successfully uninstalled openai-1.76.0


In [ ]:
import os
import pandas as pd
import openai
from dotenv import load_dotenv
import pdfkit

# Load environment variables
load_dotenv()

# Set up OpenAI API key
openai.api_key = os.getenv("OPENAI_API_KEY")

# Check if the API key is set
if not openai.api_key:
    raise ValueError("OpenAI API key is not set. Please check your .env file.")

## Data Extraction

Let's define functions to load our financial data from either a CSV file or Google Sheets.

In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

def load_from_google_sheets():
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name(os.getenv("GOOGLE_APPLICATION_CREDENTIALS"), scope)
    client = gspread.authorize(creds)

    sheet = client.open_by_url(os.getenv("DUMMY_DATA_SPREADSHEET")).sheet1
    data = sheet.get_all_records()
    return pd.DataFrame(data)

def load_from_csv():
    csv_path = '../data/data.csv'  # Assuming the CSV file is in the same directory as the notebook
    if not os.path.exists(csv_path):
        raise FileNotFoundError(f"The file {csv_path} does not exist.")
    return pd.read_csv(csv_path)

def load_data():
    if os.getenv("USE_GOOGLE_SHEETS", "false").lower() == "true":
        return load_from_google_sheets()
    else:
        return load_from_csv()

# Load the financial data
df = load_data()

# Display the first few rows of the dataframe
print(df.head())

# Display basic information about the dataframe
print(df.info())

         Date                Description       Category Income/Expense  \
0  2024-09-01       Salary from ABC Corp         Income         Income   
1  2024-09-02  Grocery shopping at Coles      Groceries        Expense   
2  2024-09-03          Sydney Water Bill      Utilities        Expense   
3  2024-09-04       Netflix Subscription  Entertainment        Expense   
4  2024-09-05   Dinner at The Rocks Cafe     Dining Out        Expense   

    Amount  
0  6000.00  
1   200.50  
2    80.75  
3    13.99  
4    60.00  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121 entries, 0 to 120
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Date            121 non-null    object 
 1   Description     121 non-null    object 
 2   Category        121 non-null    object 
 3   Income/Expense  121 non-null    object 
 4   Amount          121 non-null    float64
dtypes: float64(1), object(4)
memory usage: 4.9+ KB
None


## Transaction Categorization

Now, let's define functions to categorize the transactions using the OpenAI API.

In [ ]:
def categorize_transaction(description):
    completion = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful financial assistant. Categorize transactions into one of the following categories: Groceries, Utilities, Rent, Entertainment, Transportation, Dining Out, Miscellaneous, Health & Fitness, Housing, Investments, Insurance, Charity, and Income."},
            {"role": "user", "content": f"Categorize the following transaction: '{description}'. Respond with the category name only."}
        ]
    )
    return completion.choices[0].message.content.strip()

def categorize_transactions(df):
    for index, row in df[df['Category'] == ''].iterrows():
        category = categorize_transaction(row['Description'])
        df.at[index, 'Category'] = category
    return df

# Categorize the transactions
df = categorize_transactions(df)

# Display the first few rows of the categorized dataframe
print(df.head())

# Show the unique categories
print("Unique categories:")
print(df['Category'].unique())

         Date                Description       Category Income/Expense  \
0  2024-09-01       Salary from ABC Corp         Income         Income   
1  2024-09-02  Grocery shopping at Coles      Groceries        Expense   
2  2024-09-03          Sydney Water Bill      Utilities        Expense   
3  2024-09-04       Netflix Subscription  Entertainment        Expense   
4  2024-09-05   Dinner at The Rocks Cafe     Dining Out        Expense   

    Amount  
0  6000.00  
1   200.50  
2    80.75  
3    13.99  
4    60.00  
Unique categories:
['Income' 'Groceries' 'Utilities' 'Entertainment' 'Dining Out'
 'Transportation' 'Fitness' 'Accommodation' 'Miscellaneous' 'Investments'
 'Insurance' 'Charity' 'Shopping' 'Travel' 'Electronics' 'Expense']


## Financial Summary

Now that we have categorized our transactions, let's generate a financial summary using the OpenAI API.

In [ ]:
def generate_financial_summary(df):
    total_spent = df[df['Income/Expense'] == 'Expense']['Amount'].sum()
    total_income = df[df['Income/Expense'] == 'Income']['Amount'].sum()
    category_spending = df[df['Income/Expense'] == 'Expense'].groupby('Category')['Amount'].sum()

    completion = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful financial assistant. Generate a concise financial summary based on the provided data."},
            {"role": "user", "content": f"Generate a financial summary based on the following data:\n"
                                        f"Total Income: ${total_income:.2f}\n"
                                        f"Total Expenses: ${total_spent:.2f}\n"
                                        f"Spending by category: {category_spending.to_dict()}"}
        ]
    )
    return completion.choices[0].message.content.strip()

# Generate the financial summary
summary = generate_financial_summary(df)

print("Financial Summary:")
print(summary)

Financial Summary:
**Financial Summary**

- **Total Income:** $39,952.00
- **Total Expenses:** $22,590.21
- **Net Income:** $17,361.79 (Total Income - Total Expenses)

**Spending by Category:**
- Accommodation: $9,250.00
- Charity: $900.00
- Dining Out: $1,081.50
- Electronics: $3,500.00
- Entertainment: $581.96
- Expense: $200.00
- Fitness: $160.00
- Groceries: $2,292.00
- Insurance: $600.00
- Miscellaneous: $370.00
- Shopping: $1,205.00
- Transportation: $693.00
- Travel: $950.00
- Utilities: $806.75

**Key Insights:**
- The net income indicates a strong financial position, with income exceeding expenses by a significant margin.
- The largest spending categories are Accommodation and Electronics, which collectively account for a substantial portion of total expenses.


## Personalized Financial Advice

Let's generate some personalized financial advice based on the transaction data and some additional personal information.

In [ ]:
def generate_personalized_advice(df, age=None, lifestyle=None, hobbies=None):
    transactions = df.to_string(index=False)

    prompt = f"""
    Based on the following transaction data:

    {transactions}

    And considering the following personal information:
    Age: {age}
    Lifestyle: {lifestyle}
    Hobbies: {hobbies}

    Please provide personalized financial advice. Include suggestions for budgeting, saving, and potential areas for improvement.
    """

    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful financial advisor."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content

# Generate personalized advice
advice = generate_personalized_advice(df, age=30, lifestyle="Urban", hobbies="Reading, Traveling")

print("Personalized Financial Advice:")
print(advice)

Personalized Financial Advice:
Based on the transaction data you've provided, I will analyze your income and expenses, and then provide some personalized financial advice that aligns with your age, lifestyle, and hobbies.

### Income Overview
Your income consists of:

- Regular salary from ABC Corp: $6,000 per month
- From freelance work: $1,500 to $2,200 per month on average
- Investment dividends: Approximately $250 to $280 per quarter
- Other one-time income like gifts and bonuses.

**Total Monthly Income Estimate:**
- Monthly salary: $6,000
- Freelance income (monthly average): $1,750 (based on the data)
- Dividend income (monthly average): $70 (approximate quarterly average)

**Total Monthly Income = $6,000 + $1,750 + $70 = $7,820**

### Expense Overview
Your expenses are categorized into various areas. Based on the provided data, here’s a rough breakdown of your average monthly expenses:

1. **Housing (Rent):** $2,000
2. **Utilities:** $110 (average)
3. **Groceries:** $203
4. **T

## PDF Report Generation

Now, let's create a function to generate a PDF report containing the financial summary and personalized advice.

**Please note that this requires `wkhtmltopdf` to be installed on your system.**

In [ ]:
def create_pdf_report(summary, advice, output_path='financial_report.pdf'):
    html_content = """
    <!DOCTYPE html>
    <html>
    <head>
        <meta charset="UTF-8">
        <title>Financial Report</title>
        <style>
            body {{ font-family: Arial, sans-serif; line-height: 1.6; color: #333; }}
            h1 {{ color: #2c3e50; text-align: center; }}
            h2 {{ color: #34495e; }}
            h3 {{ color: #2980b9; }}
            .section {{ margin-bottom: 20px; }}
            ul {{ padding-left: 20px; }}
        </style>
    </head>
    <body>
        <h1>Financial Report</h1>

        <div class="section">
            <h2>Financial Summary</h2>
            {0}
        </div>

        <div class="section">
            <h2>Personalized Financial Advice</h2>
            {1}
        </div>
    </body>
    </html>
    """.format(
        summary.replace('**', '').replace('###', '<h3>').replace('\n', '<br>'),
        advice.replace('**', '').replace('###', '<h3>').replace('\n', '<br>')
    )

    options = {
        'page-size': 'A4',
        'margin-top': '0.75in',
        'margin-right': '0.75in',
        'margin-bottom': '0.75in',
        'margin-left': '0.75in',
        'encoding': "UTF-8",
    }

    pdfkit.from_string(html_content, output_path, options=options)
    print(f"PDF report generated: {output_path}")

# Generate the PDF report
create_pdf_report(summary, advice)

print("PDF report has been generated.")

PDF report generated: financial_report.pdf
PDF report has been generated.


## Conclusion

In this notebook, we've walked through the process of loading financial data, categorizing transactions, generating a financial summary, providing personalized financial advice, and creating a PDF report. All the necessary code is contained within this notebook, making it easy to run and modify without external dependencies.

You can further extend this analysis by:
1. Visualizing spending patterns using matplotlib or seaborn
2. Implementing more advanced financial metrics
3. Exploring trends over time in the financial data
4. Customizing the PDF report layout and content

Feel free to experiment with the code and adapt it to your specific financial analysis needs!